Ćwiczenie wstępne z MPI. Na podstawie:
* http://selkie.macalester.edu/DistributedPython/index.html

* https://mpi4py.readthedocs.io/en/stable/index.html

In [7]:
!pip install mpi4py

Defaulting to user installation because normal site-packages is not writeable
  Using cached mpi4py-3.1.5.tar.gz (2.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.5-cp39-cp39-linux_x86_64.whl size=639492 sha256=aa1a0898214c8e10d5d866a7144177d589971b066ba7687459bf90b44695958a
  Stored in directory: /home/stud/.cache/pip/wheels/17/26/3e/a5512b087666ce03512d7eb1b16782e8b104577c3649b68633
Successfully built mpi4py


In [8]:
%%writefile example0.py

from mpi4py import MPI

def main():
  comm = MPI.COMM_WORLD # komunikator zawierający wszystkie procesy
  id = comm.Get_rank() # id aktualnego procesu
  numProc = comm.Get_size() # liczba procesów w komunikatorze
  hostName = MPI.Get_processor_name() # nazwa maszyny na której uruchamiany jest program

  print(f"Jestem procesem nr {id} z {numProc} uruchamianym na maszynie {hostName}")

main()


Writing example0.py


In [10]:
#uruchomienie na komputrze lokalnym:
# mpirun -np 4 python example0.py 
# ! oraz opcja --allow-run-as-root dodane na potrzeby notatnika
!mpirun --allow-run-as-root -np 4 python3 example0.py

Jestem procesem nr 0 z 4 uruchamianym na maszynie ubuntu_1804_desktop
Jestem procesem nr 1 z 4 uruchamianym na maszynie ubuntu_1804_desktop
Jestem procesem nr 3 z 4 uruchamianym na maszynie ubuntu_1804_desktop
Jestem procesem nr 2 z 4 uruchamianym na maszynie ubuntu_1804_desktop


In [11]:
%%writefile example1.py

from mpi4py import MPI

def main():
  comm = MPI.COMM_WORLD # komunikator zawierający wszystkie procesy
  id = comm.Get_rank() # id aktualnego procesu
  numProc = comm.Get_size() # liczba procesów w komunikatorze
  hostName = MPI.Get_processor_name() # nazwa maszyny na której uruchamiany jest program

  if id == 0:
    print(f"Jestem procesem ** głównym ** o id {id} uruchamianym na maszynie {hostName}")
  else:
    print(f"Jestem procesem nr {id} z {numProc} uruchamianym na maszynie {hostName}")


main()

Writing example1.py


In [13]:
!mpirun --allow-run-as-root -np 4 python3 example1.py

Jestem procesem nr 2 z 4 uruchamianym na maszynie ubuntu_1804_desktop
Jestem procesem ** głównym ** o id 0 uruchamianym na maszynie ubuntu_1804_desktop
Jestem procesem nr 3 z 4 uruchamianym na maszynie ubuntu_1804_desktop
Jestem procesem nr 1 z 4 uruchamianym na maszynie ubuntu_1804_desktop


In [14]:
%%writefile example2.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    REPS = 16

    if ((REPS % numProcesses) == 0 and numProcesses <= REPS):
        # How much of the loop should a process work on?
        # Każdy proces otrzymuje kolejno odpowiednią ilość zadań określoną jako chunkSize. np. proces 0 otrzymuje 0, 1, 2, 3. ChunkSize to 4.
        chunkSize = int(REPS / numProcesses)
        start = id * chunkSize
        stop = start + chunkSize

        # do the work within the range set aside for this process
        for i in range(start, stop):
            print("On {}: Process {} is performing iteration {}"\
            .format(myHostName, id, i))

    else:
        # cannot break into equal chunks; one process reports the error
        if id == 0 :
            print("Please run with number of processes divisible by \
and less than or equal to {}.".format(REPS))

########## Run the main function
main()


Writing example2.py


In [16]:
! mpirun --allow-run-as-root -np 4 python3 example2.py

On ubuntu_1804_desktop: Process 3 is performing iteration 12
On ubuntu_1804_desktop: Process 3 is performing iteration 13
On ubuntu_1804_desktop: Process 3 is performing iteration 14
On ubuntu_1804_desktop: Process 3 is performing iteration 15
On ubuntu_1804_desktop: Process 1 is performing iteration 4
On ubuntu_1804_desktop: Process 1 is performing iteration 5
On ubuntu_1804_desktop: Process 1 is performing iteration 6
On ubuntu_1804_desktop: Process 1 is performing iteration 7
On ubuntu_1804_desktop: Process 2 is performing iteration 8
On ubuntu_1804_desktop: Process 2 is performing iteration 9
On ubuntu_1804_desktop: Process 2 is performing iteration 10
On ubuntu_1804_desktop: Process 2 is performing iteration 11
On ubuntu_1804_desktop: Process 0 is performing iteration 0
On ubuntu_1804_desktop: Process 0 is performing iteration 1
On ubuntu_1804_desktop: Process 0 is performing iteration 2
On ubuntu_1804_desktop: Process 0 is performing iteration 3



*   Proszę uruchomić powyższy program użwając różnej liczby procesów (N = 1,2,4,8);  

*   zmienić wartość REPS na 16
* wyjaśnić jak schemat dzieli iteracje pętli między procesy

In [17]:
%%writefile example3.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    REPS = 8
    #Każdy proces bierze kolejne dostępne zadanie, przez co jeśli każdy z procesów weźmie dostępne zadanie, kolejka zaczyna się od początku.
    #Na przykład Proces 0 -> zadanie 0, proces 1 -> zadanie 1..., proces 0 -> zadanie 4... 
    if (numProcesses <= REPS):

        for i in range(id, REPS, numProcesses):
            print("On {}: Process {} is performing iteration {}"\
            .format(myHostName, id, i))

    else:
        # can't have more processes than work; one process reports the error
        if id == 0 :
            print("Please run with number of processes less than \
or equal to {}.".format(REPS))

########## Run the main function
main()


Writing example3.py


In [18]:
! mpirun --allow-run-as-root -np 4 python3 example3.py

On ubuntu_1804_desktop: Process 3 is performing iteration 3
On ubuntu_1804_desktop: Process 3 is performing iteration 7
On ubuntu_1804_desktop: Process 1 is performing iteration 1
On ubuntu_1804_desktop: Process 1 is performing iteration 5
On ubuntu_1804_desktop: Process 2 is performing iteration 2
On ubuntu_1804_desktop: Process 2 is performing iteration 6
On ubuntu_1804_desktop: Process 0 is performing iteration 0
On ubuntu_1804_desktop: Process 0 is performing iteration 4


* Proszę uruchomić powyższy program użwając różnej liczby procesów (N = 1,2,4,8);

* zmienić wartość REPS na 16

* wyjaśnić jak schemat dzieli iteracje pętli między procesy

Komunikacja punkt-punkt

In [19]:
%%writefile example4.py
from mpi4py import MPI

# function to return whether a number of a process is odd or even
def odd(number):
    if (number % 2) == 0:
        return False
    else :
        return True

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 and not odd(numProcesses):
        #generate a list of 8 numbers, beginning with my id
        sendList = list(range(id, id+8))
        if odd(id):
            #odd processes send to their 'left neighbor', then receive from
            comm.send(sendList, dest=id-1)
            receivedList = comm.recv(source=id-1)
        else :
            #even processes receive from their 'right neighbor', then send
            receivedList = comm.recv(source=id+1)
            comm.send(sendList, dest=id+1)

        print("Process {} of {} on {} computed {} and received {}"\
        .format(id, numProcesses, myHostName, sendList, receivedList))

    else :
        if id == 0:
            print("Please run this program with the number of processes \
positive and even")

########## Run the main function
main()

Writing example4.py


In [20]:
! mpirun --allow-run-as-root -np 4 python3 example4.py

Process 0 of 4 on ubuntu_1804_desktop computed [0, 1, 2, 3, 4, 5, 6, 7] and received [1, 2, 3, 4, 5, 6, 7, 8]
Process 1 of 4 on ubuntu_1804_desktop computed [1, 2, 3, 4, 5, 6, 7, 8] and received [0, 1, 2, 3, 4, 5, 6, 7]
Process 2 of 4 on ubuntu_1804_desktop computed [2, 3, 4, 5, 6, 7, 8, 9] and received [3, 4, 5, 6, 7, 8, 9, 10]
Process 3 of 4 on ubuntu_1804_desktop computed [3, 4, 5, 6, 7, 8, 9, 10] and received [2, 3, 4, 5, 6, 7, 8, 9]


Proszę narysować schemat komunikacji pomiędzy poszczególnymi procesami w zadaniu numer 4

##Zadanie
**Zadanie.** Proszę napisać program który generuje tablicę tablic liczb w procesie głównym i rozsyłą je do pozostałych procesów. Procesy odbierają swoją część tablicy i wyliczają oraz wypisują jej sumę;   

In [55]:
%%writefile zadanie1.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()    
    numSideProcesses = comm.Get_size() - 1
    myHostName = MPI.Get_processor_name()

    if numSideProcesses > 1:
        if id == 0:
            sendList = list(range(id, id+8))
            for i in range(numSideProcesses):
                start = int(i * len(sendList) / numSideProcesses)
                end = int(start + len(sendList) / numSideProcesses)
                comm.send(sendList[start:end], dest=i+1)
        else:
            receivedList = comm.recv(source=0)
            result =0
            for element in receivedList:
                result+=element
            print("Jestem procesem o id:", id)
            print("Obliczyłem sumę:", result)

main()

Overwriting zadanie1.py


In [56]:
!mpirun --allow-run-as-root -np 5 python3 zadanie1.py

Jestem procesem o id: 4
Jestem procesem o id: 3
Jestem procesem o id: 2
Obliczyłem sumę: 5
Obliczyłem sumę: 9
Obliczyłem sumę: 13
Jestem procesem o id: 1
Obliczyłem sumę: 1


Komunikacja kolektywna

In [21]:
%%writefile broadcast_example.py
from mpi4py import MPI

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    if numProcesses > 1 :

        if id == 0:        # master
            #master: generate a dictionary with arbitrary data in it
            data = list(range(numProcesses))
            print("Master Process {} of {} on {} broadcasts {}"\
            .format(id, numProcesses, myHostName, data))

        else :
            # worker: start with empty data
            data = []
            print("Worker Process {} of {} on {} starts with {}"\
            .format(id, numProcesses, myHostName, data))

        #initiate and complete the broadcast
        data = comm.bcast(data, root=0)

        #check the result
        print("Process {} of {} on {} has {} after the broadcast"\
        .format(id, numProcesses, myHostName, data))

    else :
        print("Please run this program with the number of processes greater than 1")

########## Run the main function
main()


Writing broadcast_example.py


In [22]:
! mpirun --allow-run-as-root -np 4 python3 broadcast_example.py

Worker Process 3 of 4 on ubuntu_1804_desktop starts with []
Master Process 0 of 4 on ubuntu_1804_desktop broadcasts [0, 1, 2, 3]
Process 0 of 4 on ubuntu_1804_desktop has [0, 1, 2, 3] after the broadcast
Worker Process 2 of 4 on ubuntu_1804_desktop starts with []
Process 2 of 4 on ubuntu_1804_desktop has [0, 1, 2, 3] after the broadcast
Worker Process 1 of 4 on ubuntu_1804_desktop starts with []
Process 1 of 4 on ubuntu_1804_desktop has [0, 1, 2, 3] after the broadcast
Process 3 of 4 on ubuntu_1804_desktop has [0, 1, 2, 3] after the broadcast


Scatter, Gather 

In [ ]:
%%writefile 14scatter.py
from mpi4py import MPI

# Create a list of lists to be scattered.
def genListOfLists(numElements):
    data = [[0]*3 for i in range(numElements)]
    for i in range(numElements):
        #make small lists of 3 distinct elements
        smallerList = []
        for j in range(1,4):
            smallerList = smallerList + [(i+1)*j]
        # place the small list in the larger list
        data[i] = smallerList
    return data

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    # in mpi4py, the lowercase scatter method only works on lists whose size
    # is the total number of processes.
    numElements = numProcesses      #total elements in list created by master process

    # however, the list can contain lists, like this list of 3-element lists,
    # for example this list of four 3-element lists:
    #     [[1, 2, 3], [2, 4, 6], [3, 6, 9], [4, 8, 12]]

    if id == 0:
        data = genListOfLists(numElements)
        print("Master {} of {} on {} has created list: {}"\
        .format(id, numProcesses, myHostName, data))
    else:
        data = None
        print("Worker Process {} of {} on {} starts with {}"\
        .format(id, numProcesses, myHostName, data))

    #scatter one small list in the large list on node 0 to each of the processes
    result = comm.scatter(data, root=0)

    print("Process {} of {} on {} has result after scatter {}"\
    .format(id, numProcesses, myHostName, result))

    if id == 0:
        print("Master {} of {} on {} has original list after scatter: {}"\
        .format(id, numProcesses, myHostName, data))

########## Run the main function
main()

Overwriting 14scatter.py


In [49]:
! mpirun --allow-run-as-root -np 8 python3 14scatter.py

Worker Process 6 of 8 on ubuntu_1804_desktop starts with None
Worker Process 3 of 8 on ubuntu_1804_desktop starts with None
Master 0 of 8 on ubuntu_1804_desktop has created list: [[1, 2, 3], [2, 4, 6], [3, 6, 9], [4, 8, 12], [5, 10, 15], [6, 12, 18], [7, 14, 21], [8, 16, 24]]
Process 3 of 8 on ubuntu_1804_desktop has result after scatter [4, 8, 12]
Process 0 of 8 on ubuntu_1804_desktop has result after scatter [1, 2, 3]
Master 0 of 8 on ubuntu_1804_desktop has original list after scatter: [[1, 2, 3], [2, 4, 6], [3, 6, 9], [4, 8, 12], [5, 10, 15], [6, 12, 18], [7, 14, 21], [8, 16, 24]]
Process 6 of 8 on ubuntu_1804_desktop has result after scatter [7, 14, 21]
Worker Process 1 of 8 on ubuntu_1804_desktop starts with None
Process 1 of 8 on ubuntu_1804_desktop has result after scatter [2, 4, 6]
Worker Process 2 of 8 on ubuntu_1804_desktop starts with None
Worker Process 5 of 8 on ubuntu_1804_desktop starts with None
Worker Process 7 of 8 on ubuntu_1804_desktop starts with None
Worker Proce

Zadanie. Proszę napisać program który generuje tablicę tablic liczb w procesie głównym i rozsyłą je do pozostałych procesów. Procesy odbierają swoją część tablicy i wyliczają oraz wypisują jej sumę. Proszę napisac program z wykorzystaniem komunikacji kolektywnej

In [53]:
%%writefile zadanie2.py
from mpi4py import MPI

# Create a list of lists to be scattered.
def genListOfLists(numElements):
    data = [[0]*3 for i in range(numElements)]
    for i in range(numElements):
        #make small lists of 3 distinct elements
        smallerList = []
        for j in range(1,4):
            smallerList = smallerList + [(i+1)*j]
        # place the small list in the larger list
        data[i] = smallerList
    return data

def main():
    comm = MPI.COMM_WORLD
    id = comm.Get_rank()            #number of the process running the code
    numProcesses = comm.Get_size()  #total number of processes running
    myHostName = MPI.Get_processor_name()  #machine name running the code

    # in mpi4py, the lowercase scatter method only works on lists whose size
    # is the total number of processes.
    numElements = numProcesses      #total elements in list created by master process

    # however, the list can contain lists, like this list of 3-element lists,
    # for example this list of four 3-element lists:
    #     [[1, 2, 3], [2, 4, 6], [3, 6, 9], [4, 8, 12]]

    if id == 0:
        data = genListOfLists(numElements)
        print("Master {} of {} on {} has created list: {}"\
        .format(id, numProcesses, myHostName, data))
    else:
        data = None
        print("Worker Process {} of {} on {} starts with {}"\
        .format(id, numProcesses, myHostName, data))

    #scatter one small list in the large list on node 0 to each of the processes
    list_result = comm.scatter(data, root=0)

    #print("Process {} of {} on {} has result after scatter {}"\
    #.format(id, numProcesses, myHostName, result))
    result = 0
    for element in list_result:
        result+= element

    print("Proces {} z {} na {} wyliczył sumę tablicy {}"\
    .format(id, numProcesses, myHostName, result))

    if id == 0:
        print("Master {} of {} on {} has original list after scatter: {}"\
        .format(id, numProcesses, myHostName, data))

########## Run the main function
main()

Overwriting zadanie2.py


In [57]:
! mpirun --allow-run-as-root -np 8 python3 zadanie2.py

Worker Process 7 of 8 on ubuntu_1804_desktop starts with None
Worker Process 3 of 8 on ubuntu_1804_desktop starts with None
Worker Process 4 of 8 on ubuntu_1804_desktop starts with None
Worker Process 5 of 8 on ubuntu_1804_desktop starts with None
Worker Process 1 of 8 on ubuntu_1804_desktop starts with None
Master 0 of 8 on ubuntu_1804_desktop has created list: [[1, 2, 3], [2, 4, 6], [3, 6, 9], [4, 8, 12], [5, 10, 15], [6, 12, 18], [7, 14, 21], [8, 16, 24]]
Worker Process 6 of 8 on ubuntu_1804_desktop starts with None
Worker Process 2 of 8 on ubuntu_1804_desktop starts with None
Proces 5 z 8 na ubuntu_1804_desktop wyliczył sumę tablicy 36
Proces 0 z 8 na ubuntu_1804_desktop wyliczył sumę tablicy 6
Proces 3 z 8 na ubuntu_1804_desktop wyliczył sumę tablicy 24
Proces 4 z 8 na ubuntu_1804_desktop wyliczył sumę tablicy 30
Proces 2 z 8 na ubuntu_1804_desktop wyliczył sumę tablicy 18
Proces 6 z 8 na ubuntu_1804_desktop wyliczył sumę tablicy 42
Proces 1 z 8 na ubuntu_1804_desktop wyliczył sum